In [1]:
import numpy as np
from synd.core import load_model
import deeptime.markov.msm as dm

Load the SynD model, and the source/target state definitions

In [2]:
model = load_model('coarser-model.synd')
state_definitions = np.load('coarser-state_definitions.npz')

In [3]:
source_states = state_definitions['source']
target_states = state_definitions['target']

# Generate sample trajectories

In [4]:
initial_points = np.concatenate([source_states for x in range(10)])
initial_points

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Seed the RNG for reproducible outputs

In [5]:
model.rng = np.random.default_rng(seed=50)

Generate a 1000-step trajectory, from our set of 30 source states

In [6]:
sample_trajectories = model.generate_trajectory(
    initial_states = initial_points,
    n_steps = 10000,
)

In [7]:
sample_trajectories

array([[ 0,  9, 10, ..., 62, 62, 62],
       [ 0,  5,  6, ..., 99, 99, 51],
       [ 0,  6,  7, ..., 62, 38, 56],
       ...,
       [ 0, 11, 12, ..., 76, 95, 72],
       [ 0,  6,  8, ..., 89, 89, 58],
       [ 0,  6,  6, ..., 61, 99, 86]])

Check that the sample trajectories actually reached the target state

In [8]:
np.isin(target_states, sample_trajectories).any()

True

In [9]:
np.save('coarser-sample_trajectories.npy', sample_trajectories)

# Generate reference equilibrium / NESS

In [10]:
ref_msm = dm.MarkovStateModel(model.transition_matrix)
ref_equil = ref_msm.stationary_distribution

In [11]:
ref_recycling_tmatrix = model.transition_matrix.copy()

ref_recycling_tmatrix[target_states,:] = 0.0
ref_recycling_tmatrix[target_states, source_states] = 1.0

ref_msm = dm.MarkovStateModel(ref_recycling_tmatrix)
ref_ness = ref_msm.stationary_distribution

In [12]:
np.savez('coarser-reference_distributions.npz', equil=ref_equil, ness=ref_ness)